# PHD Deepak2880 Process Data on Test Data

# Preprocess on Test Data

####  $ Long $ Term $ Stock $ Price $ Growth $ Prediction $ using $ NLP $ on $ 10-K $ Financial $ Reports 

# 1. Bussiness Understanding

A 10-K FInancial Report is a comprehensive report which must be filed annually by
all publicly traded companies about its financial performance. These reports are filed
to the US Securities Exchange Commission (SEC). This is even more detailed than
the annual report of a company. The 10K documents contain information about the
Business' operations, risk factors, selected financial data, the Management's
discussion and analysis (MD&A) and also Financial Statements and supplementary
data. 10-K reports are very important for investors and Warren Buffet consistently
cites these reports as a great source of information about a company's potential to
succeed.

In this problem statement we have to Extract the 10-K FORM from the given "10k_filing_info". Once we get all the 10-K FORM then we have to predict the long term stock performance of a company from the 10-K docs.

# 2. Problem Faced 

## 2.1 Data Extraction

Major Problem faced during the Extraction of the 10-K FORM was the size of file in the URL. Size of file is around 115MB for each, Also each URL contains approximately 8-12 documents, out of those doucements I have to extract only 10-K FORMS.I cracked this using soup library.
Now new problem was waiting. There are around 2500 URL present. I Have to extract 10-K FORM for each URL. FIrst I tried only on 10-15 URL. It was taking around 20-25 minutes, then I decided that within a single loop its difficult to extract all the 10-K FORM also there are high chances of getting my IP ADDRESS blocked as all the url are sitting on the US GOV. server. They can find my activity as DDOS. So finally I decided to go with 1000 URL at a time. For downloading 1000 URL and extract 10-K FORM it took around 4 HRS. I tried to use multiprocessing.pool library to extract at a very few mins but due to DDOS I did not use this mehtod.

## 2.2 Feature Extraction

After Extracting all the 10-K FORM new problem is waiting. Problem was to extract only given feature from the 10-K FORM. I almost search every file thst I have extracted for patterns. But in such limited span of time I could find only some features. Then by using Regular Expression I extracted the desired data from the 10-Kform. I am able to get around 30-40% feature only.
I will improve my Reguar expresions to get more feature.

  # 3 $DATA$ $PREPROCESSING$

## 3.1. Load the Module to Read the CSV File

In [1]:
#pip install swifter

In [2]:
import os 
import re
import pandas as pd
import time
import numpy as np
import swifter

In [3]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [4]:
%%time
#data = pd.read_csv("/content/drive/My Drive/PHD/PHD_Test_Data.csv")
data = pd.read_csv("PHD_Test_Data.csv")

Wall time: 3.19 s


In [5]:
data.head()

,ticker,cik,filing_date,sentence,id
0,GD,40533,2/17/2012,['1\nd271667d10k.htm\nFORM 10-K\n\n\nForm 10-K...,0
1,MTB,36270,3/19/1999,['1\n10-K\n\n\n\n\n ...,1
2,RRC,315852,3/6/2001,"[""1\nl86830ae10-k.htm\nRANGE RESOURCES CORPORA...",2
3,LH,920148,2/26/2009,"[""1\nlabcorp10k.htm\n\n\n\n \n\n\nUNITED STATE...",3
4,SYY,96021,9/16/2004,['1\nh18314e10vk.txt\nSYSCO CORPORATION - JULY...,4


In [6]:
wdata = data.iloc[:,3:].copy()
#del data_1,data_2,data_3

In [7]:
wdata.head()

,sentence,id
0,['1\nd271667d10k.htm\nFORM 10-K\n\n\nForm 10-K...,0
1,['1\n10-K\n\n\n\n\n ...,1
2,"[""1\nl86830ae10-k.htm\nRANGE RESOURCES CORPORA...",2
3,"[""1\nlabcorp10k.htm\n\n\n\n \n\n\nUNITED STATE...",3
4,['1\nh18314e10vk.txt\nSYSCO CORPORATION - JULY...,4


In [8]:
'''%%time
wdata["clean_text"] = wdata["sentence"].apply(remove_html_tags)'''

'%%time\nwdata["clean_text"] = wdata["sentence"].apply(remove_html_tags)'

In [9]:
wdata.head()

,sentence,id
0,['1\nd271667d10k.htm\nFORM 10-K\n\n\nForm 10-K...,0
1,['1\n10-K\n\n\n\n\n ...,1
2,"[""1\nl86830ae10-k.htm\nRANGE RESOURCES CORPORA...",2
3,"[""1\nlabcorp10k.htm\n\n\n\n \n\n\nUNITED STATE...",3
4,['1\nh18314e10vk.txt\nSYSCO CORPORATION - JULY...,4


# 4. FEATURE EXTRACTION Regular Expression

In [10]:
look_up = {'BUSINESS': r'ITEM.{,80}BUSINESS(.*?)ITEM.?\d',
          'RISK FACTORS': r'ITEM.{,80}RISK.{,3}FACTORS(.*?)ITEM.?\d',
          'SELECTED FINANCIAL DATA': r'ITEM.{,80}SELECTED.{,3}(?:CONSOLIDATED.{,3})?FINANCIAL.{,3}DATA(.*?)ITEM.?\d',
          'MDA': r'ITEM.{,80}MANAGEMENT.*DISCUSSION.{,3}AND.{,3}ANALYSIS.{,3}OF.{,3}FINANCIAL.{,3}CONDITION.{,3}AND.{,3}RESULTS.{,3}OF.{,3}OPERATIONS(.*?)ITEM.?\d',
          'FSS': r'ITEM.{,80}FINANCIAL.*STATEMENTS.{,3}AND.{,3}SUPPLEMENTARY.{,3}DATA(.*?)ITEM.?\d'}

In [11]:
def update_filter_dic(data):
    if filter_dic.get('BUSINESS'):
        b = filter_dic.get('BUSINESS')
        r = filter_dic.get('RISK FACTORS')
        s = filter_dic.get('SELECTED FINANCIAL DATA')
        m = filter_dic.get('MDA')
        f = filter_dic.get('FSS')
    else:
        b,r,s,m,f = [],[],[],[],[]
    b.append(data['BUSINESS'])
    r.append(data['RISK FACTORS'])
    s.append(data['SELECTED FINANCIAL DATA'])
    m.append(data['MDA'])
    f.append(data['FSS'])
    filter_dic['BUSINESS'] = b.copy()
    filter_dic['RISK FACTORS'] = r.copy()
    filter_dic['SELECTED FINANCIAL DATA'] = s.copy()
    filter_dic['MDA'] = m.copy()
    filter_dic['FSS'] = f.copy()
print("Updated Sucessfully!")

Updated Sucessfully!


In [12]:
def parse_data(data_web):
    return_data = {}
    data_web = data_web.replace("xa0",'')
    for item,pattern in look_up.items():
        return_data[item]=''
        filter_data=re.findall(pattern,data_web,re.IGNORECASE | re.DOTALL | re.MULTILINE)
        if item == "BUSINESS":
          if len(filter_data)>=1:
            return_data[item] = filter_data
          else:
            if len(data_web)>=30000:
              return_data[item] = [data_web[5000:15000]]
            else :
              return_data[item] = [data_web[0:3000]]
        elif item == 'RISK FACTORS':
          if len(filter_data)>=1:
            return_data[item] = filter_data
          else:
            if len(data_web)>=30000:
              return_data[item] = [data_web[5000:15000]]
            else:
              return_data[item] = [data_web[3000:6000]]
        elif item == 'SELECTED FINANCIAL DATA':
          if len(filter_data)>=1:
            return_data[item] = filter_data
          else:
            if len(data_web)>=10000:
              return_data[item] = [data_web[8000:12000]]
            else:
              return_data[item] = [data_web[3000:6000]]  
        elif item == 'MDA':
          if len(filter_data)>=1:
            return_data[item] = filter_data
          else:
            if len(data_web)>=10000:
              return_data[item] = [data_web[8000:12000]]
            else:
              return_data[item] = [data_web[6000:8000]] 
        else :
          if len(filter_data)>=1:
            return_data[item] = filter_data
          else:
            if len(data_web)>=10000:
              return_data[item] = [data_web[8000:12000]]
            else:
              return_data[item] = [data_web[3000:6000]]               

    return return_data

In [13]:
%%time
g= []
for t in wdata["sentence"]:
    tm = parse_data(t)
    g.append(tm.copy())

Wall time: 36.9 s


In [14]:
filter_dic = {'BUSINESS':None,'RISK FACTORS':None,'SELECTED FINANCIAL DATA':None,'MDA':None,'FSS':None}

In [15]:
for k in g:
    update_filter_dic(k)

In [16]:
ext_data = pd.DataFrame(filter_dic)

In [18]:
ext_data.head()

,BUSINESS,RISK FACTORS,SELECTED FINANCIAL DATA,MDA,FSS
0,"[\n\\\n\\n3\n\\\n\n , OVERVIEW General Dynam...","[\n\\\n\\n14\n\\\n\n , An investment in our ...","[\n\\\n\\n18\n\\\n\n , The following table p...",[ is based on our Consolidated Financial State...,[ CONSOLIDATED STATEMENT OF EARNINGS\n \\n\n\...
1,[. ...,[ 58\n\n C. Risk elements\n ...,[.\n\n A. Selected consolidated yea...,"["". The Company\'s international activities ar...","[.""\n\n Financial statement..."
2,"[\n, \nGeneral\n\\\\\\Range Resources Corporat...",[ovides financing to small oil and gas produce...,"[\n, \n\\\\\\The following table presents sele...",[ (Capitalized terms herein are defined in the...,[\n\\\\\\Reference is made to the Index to Fin...
3,[ \n\n4 \n\n\n\n\n\n\ \n\nThe Clinical Labora...,"[ \n\n22 \n\n\n\n\n\n, \n\\nRisks Associated...","[ \n\n34 \n\n\n\n\n\n, presented below under...","[,” all included elsewhere herein.\n\\n\n\n\nY...",[ \n\n\\n\n\n\n\n\ \n\nReference is made to t...
4,[................................................,[1\nItem 7A. Quantitative and Qualitative Dis...,[..................................... 10\...,[mical products distributor\nReed Distributors...,[\n\n SYSCO CORPORATION ...


 # 5. Merge the Original Data and Analysis Filter

In [19]:
final =pd.concat([data,ext_data],axis=1)

In [20]:
final.head()

,ticker,cik,filing_date,sentence,id,BUSINESS,RISK FACTORS,SELECTED FINANCIAL DATA,MDA,FSS
0,GD,40533,2/17/2012,['1\nd271667d10k.htm\nFORM 10-K\n\n\nForm 10-K...,0,"[\n\\\n\\n3\n\\\n\n , OVERVIEW General Dynam...","[\n\\\n\\n14\n\\\n\n , An investment in our ...","[\n\\\n\\n18\n\\\n\n , The following table p...",[ is based on our Consolidated Financial State...,[ CONSOLIDATED STATEMENT OF EARNINGS\n \\n\n\...
1,MTB,36270,3/19/1999,['1\n10-K\n\n\n\n\n ...,1,[. ...,[ 58\n\n C. Risk elements\n ...,[.\n\n A. Selected consolidated yea...,"["". The Company\'s international activities ar...","[.""\n\n Financial statement..."
2,RRC,315852,3/6/2001,"[""1\nl86830ae10-k.htm\nRANGE RESOURCES CORPORA...",2,"[\n, \nGeneral\n\\\\\\Range Resources Corporat...",[ovides financing to small oil and gas produce...,"[\n, \n\\\\\\The following table presents sele...",[ (Capitalized terms herein are defined in the...,[\n\\\\\\Reference is made to the Index to Fin...
3,LH,920148,2/26/2009,"[""1\nlabcorp10k.htm\n\n\n\n \n\n\nUNITED STATE...",3,[ \n\n4 \n\n\n\n\n\n\ \n\nThe Clinical Labora...,"[ \n\n22 \n\n\n\n\n\n, \n\\nRisks Associated...","[ \n\n34 \n\n\n\n\n\n, presented below under...","[,” all included elsewhere herein.\n\\n\n\n\nY...",[ \n\n\\n\n\n\n\n\ \n\nReference is made to t...
4,SYY,96021,9/16/2004,['1\nh18314e10vk.txt\nSYSCO CORPORATION - JULY...,4,[................................................,[1\nItem 7A. Quantitative and Qualitative Dis...,[..................................... 10\...,[mical products distributor\nReed Distributors...,[\n\n SYSCO CORPORATION ...


In [21]:
final.isna().sum()

ticker                     0
cik                        0
filing_date                0
sentence                   0
id                         0
BUSINESS                   0
RISK FACTORS               0
SELECTED FINANCIAL DATA    0
MDA                        0
FSS                        0
dtype: int64

In [22]:
final.shape

(840, 10)

In [23]:
final.columns

Index(['ticker', 'cik', 'filing_date', 'sentence', 'id', 'BUSINESS',
       'RISK FACTORS', 'SELECTED FINANCIAL DATA', 'MDA', 'FSS'],
      dtype='object')

In [24]:
%%time
#final.to_csv("/content/drive/My Drive/PHD/Test_Data_With_Sentence.csv",index=False)
final.to_csv("Test_Data_With_Sentence.csv",index=False)

Wall time: 8.05 s


In [25]:
final.drop(columns=['sentence'],inplace=True)

In [26]:
%%time
#final.to_csv("/content/drive/My Drive/PHD/Test_Data.csv",index=False)
final.to_csv("Test_Data.csv",index=False)

Wall time: 2.48 s


In [27]:
train_data=final.loc[(final['BUSINESS']!="") & (final['RISK FACTORS']!="") & (final['SELECTED FINANCIAL DATA']!="") & (final['MDA']!="") & (final['FSS']!="")]

In [29]:
train_data.head()

,ticker,cik,filing_date,id,BUSINESS,RISK FACTORS,SELECTED FINANCIAL DATA,MDA,FSS
0,GD,40533,2/17/2012,0,"[\n\\\n\\n3\n\\\n\n , OVERVIEW General Dynam...","[\n\\\n\\n14\n\\\n\n , An investment in our ...","[\n\\\n\\n18\n\\\n\n , The following table p...",[ is based on our Consolidated Financial State...,[ CONSOLIDATED STATEMENT OF EARNINGS\n \\n\n\...
1,MTB,36270,3/19/1999,1,[. ...,[ 58\n\n C. Risk elements\n ...,[.\n\n A. Selected consolidated yea...,"["". The Company\'s international activities ar...","[.""\n\n Financial statement..."
2,RRC,315852,3/6/2001,2,"[\n, \nGeneral\n\\\\\\Range Resources Corporat...",[ovides financing to small oil and gas produce...,"[\n, \n\\\\\\The following table presents sele...",[ (Capitalized terms herein are defined in the...,[\n\\\\\\Reference is made to the Index to Fin...
3,LH,920148,2/26/2009,3,[ \n\n4 \n\n\n\n\n\n\ \n\nThe Clinical Labora...,"[ \n\n22 \n\n\n\n\n\n, \n\\nRisks Associated...","[ \n\n34 \n\n\n\n\n\n, presented below under...","[,” all included elsewhere herein.\n\\n\n\n\nY...",[ \n\n\\n\n\n\n\n\ \n\nReference is made to t...
4,SYY,96021,9/16/2004,4,[................................................,[1\nItem 7A. Quantitative and Qualitative Dis...,[..................................... 10\...,[mical products distributor\nReed Distributors...,[\n\n SYSCO CORPORATION ...


# 6. Feature Creation

In [30]:
df = train_data.copy()

In [31]:
df.dtypes

ticker                     object
cik                         int64
filing_date                object
id                          int64
BUSINESS                   object
RISK FACTORS               object
SELECTED FINANCIAL DATA    object
MDA                        object
FSS                        object
dtype: object

In [32]:
cat_col = df.select_dtypes(include="object").columns

In [33]:
cat_col[2:]

Index(['BUSINESS', 'RISK FACTORS', 'SELECTED FINANCIAL DATA', 'MDA', 'FSS'], dtype='object')

In [34]:
df.dtypes

ticker                     object
cik                         int64
filing_date                object
id                          int64
BUSINESS                   object
RISK FACTORS               object
SELECTED FINANCIAL DATA    object
MDA                        object
FSS                        object
dtype: object

In [35]:
df[cat_col[2:]] = df[cat_col[2:]].astype(str)

In [36]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import RegexpTokenizer, sent_tokenize
stopwords_english = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\deepa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\deepa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\deepa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
from nltk.corpus import wordnet
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [43]:
'''positiveWordsFile = r'/content/drive/My Drive/PHD/PositiveWords.txt'
nagitiveWordsFile = r'/content/drive/My Drive/PHD/NegativeWords.txt'
uncertainty_dictionaryFile = r'/content/drive/My Drive/PHD/uncertainty_dictionary.txt'''
positiveWordsFile = r'PositiveWords.txt'
nagitiveWordsFile = r'NegativeWords.txt'
uncertainty_dictionaryFile = r'uncertainty_dictionary.txt'

In [44]:
# Loading positive words
with open(positiveWordsFile,'r') as posfile:
    positivewords=posfile.read().lower()
positiveWordList=positivewords.split('\n')

In [45]:
# Loading negative words
with open(nagitiveWordsFile ,'r') as negfile:
    negativeword=negfile.read().lower()
negativeWordList=negativeword.split('\n')

In [46]:
# calculating uncertainty_score
with open(uncertainty_dictionaryFile ,'r') as uncertain_dict:
    uncertainDict=uncertain_dict.read().lower()
uncertainDictionary = uncertainDict.split('\n')

In [47]:
# Calculating positive score 
def positive_score(text):
    numPosWords = 0
    #ss = token_lemt(text)
    rawToken = tokenizer(text)
    
    for word in rawToken:
        if word in positiveWordList:
            numPosWords  += 1
    
    sumPos = numPosWords
    return sumPos

In [48]:
# Calculating Negative score
def negative_word(text):
    numNegWords=0
    #ss = token_lemt(text)
    rawToken = tokenizer(text)
    
    for word in rawToken:
        if word in negativeWordList:
            numNegWords -=1
    sumNeg = numNegWords 
    sumNeg = sumNeg * -1
    return sumNeg

In [49]:
# Calculating polarity score
def polarity_score(positiveScore, negativeScore):
    pol_score = (positiveScore - negativeScore) / ((positiveScore + negativeScore) + 0.000001)
    return pol_score

In [50]:
def uncertainty_score(text):
    uncertainWordnum =0
    #ss = token_lemt(text)
    rawToken = tokenizer(text)
    
    for word in rawToken:
        if word in uncertainDictionary:
            uncertainWordnum +=1
    sumUncertainityScore = uncertainWordnum 
    
    return sumUncertainityScore

In [51]:
# Tokenizer
def tokenizer(text):
    #text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopwords_english, tokens))
    return filtered_words

In [52]:
def clean_data(bdata):
    bdata = str(bdata)
    var = re.sub(r"[&|\,|\.|\?|\!|\(|\)|\n|\/]","",bdata).strip()
    var = re.sub(r"[\’]","",var.lower())
    var = re.sub(r"[\“\”\$\`\`\''\``]","",var)
    var = var.replace("''","")
    var = var.replace("``","")
    var = re.sub(r"[\-]","",var)
    var = re.sub(r"[\d+]","",var)
    var = re.sub(r"[\*]{,8}","",var)
    var = re.sub(r"[\—]","",var)
    var = re.sub(r"[\:]","",var)
    var = re.sub(r"[\;\%\-]","",var)
    var = re.sub("[^a-zA-z0-9\s']"," ", var)
    var = re.sub("[']","", var)
    var = re.sub("[\\\\]","", var)
    var = re.sub(r"mw","", var)
    var = re.sub(r"``","", var)
    var = re.sub(r"\n","", var)
    var = re.sub(r"[nn]{2,20}","",var)
    var = re.sub(r"['–']{,20}","",var)
    var = re.sub(r"['q']{2,20}","",var)
    var = re.sub(r"['[[']{1,20}","",var)
    var = re.sub(r"[\]\]]","",var).strip()
    return var

In [53]:
def token_lemt(var):
    v = clean_data(var)
    words = word_tokenize(v)
    words = [word for word in words if word not in stopwords_english]
    #lemma_words = [lemmatizer.lemmatize(i, get_wordnet_pos(i)) for i in words]
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(i) for i in words]
    filtered = ' '.join(lemma_words)
    return filtered

In [54]:
def token_lemt_token_only(var):
    #v = clean_data(var)
    words = word_tokenize(var)
    #words = [word for word in var if word not in stopwords_english]
    #lemma_words = [lemmatizer.lemmatize(i, get_wordnet_pos(i)) for i in words]
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(i) for i in words]
    #filtered = ' '.join(lemma_words)
    return lemma_words

In [55]:
# Calculating Average sentence length 
# It will calculated using formula --- Average Sentence Length = the number of words / the number of sentences
     
def average_sentence_length(text):
  #ss = token_lemt(text)
  sentence_list = sent_tokenize(text)
  tokens = tokenizer(text)
  totalWordCount = len(tokens)
  totalSentences = len(sentence_list)
  average_sent = 0
  if totalSentences != 0:
    average_sent = totalWordCount / totalSentences
  
  average_sent_length= average_sent
  
  return round(average_sent_length)

In [56]:
#Counting total words

def total_word_count(text):
    #ss = token_lemt(text)
    tokens = tokenizer(text)
    return len(tokens)

In [57]:
# Calculating percentage of complex word 
# It is calculated using Percentage of Complex words = the number of complex words / the number of words 

def percentage_complex_word(text):
    #ss = token_lemt(text)
    tokens = tokenizer(text)
    complexWord = 0
    complex_word_percentage = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    if len(tokens) != 0:
        complex_word_percentage = complexWord/len(tokens)
    
    return complex_word_percentage

In [58]:
# Counting complex words
def complex_word_count(text):
    #ss = token_lemt(text)
    tokens = tokenizer(text)
    complexWord = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord

In [59]:
# calculating Fog Index 
# Fog index is calculated using -- Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

def fog_index(averageSentenceLength, percentageComplexWord):
    fogIndex = 0.4 * (averageSentenceLength + percentageComplexWord)
    return fogIndex

In [ ]:
#df['A'][0:645] = df['BUSINESS'][:645].swifter.apply(token_lemt)

In [ ]:
#df['A'][646:] = df['BUSINESS'][646:].swifter.apply(token_lemt)

In [60]:
df.drop([645],axis=0,inplace = True)

In [ ]:
'''%%time
df['BUSINESS'] = df['BUSINESS'].swifter.apply(token_lemt)'''

In [62]:
%%time
#df['BUSINESS'] = df['BUSINESS'].swifter.apply(token_lemt)
df['BUSINESS'] = df['BUSINESS'].swifter.apply(token_lemt)
df['RISK FACTORS'] = df['RISK FACTORS'].swifter.apply(token_lemt)
df['SELECTED FINANCIAL DATA'] = df['SELECTED FINANCIAL DATA'].swifter.apply(token_lemt)
df['MDA'] = df['MDA'].swifter.apply(token_lemt)
df['FSS'] = df['FSS'].swifter.apply(token_lemt)


Wall time: 2min 44s


In [63]:
%%time
df['BUSINESS_Tokens'] = df['BUSINESS'].swifter.apply(token_lemt_token_only)
df['RISK_Tokens'] = df['RISK FACTORS'].swifter.apply(token_lemt_token_only)
df['SELECTED_Tokens'] = df['SELECTED FINANCIAL DATA'].swifter.apply(token_lemt_token_only)
df['MDA_Tokens'] = df['MDA'].swifter.apply(token_lemt_token_only)
df['FSS_Tokens'] = df['FSS'].swifter.apply(token_lemt_token_only)


Wall time: 1min 8s


In [64]:
%%time
df['BUSINESS_POSITIVE'] = df['BUSINESS'].swifter.apply(positive_score)
df['RISK FACTORS_POSITIVE'] = df['RISK FACTORS'].swifter.apply(positive_score)
df['SELECTED FINANCIAL DATA_POSITIVE'] = df['SELECTED FINANCIAL DATA'].swifter.apply(positive_score)
df['MDA_POSITIVE'] = df['MDA'].swifter.apply(positive_score)
df['FSS_POSITIVE'] = df['FSS'].swifter.apply(positive_score)


Wall time: 48.6 s


In [65]:
%%time
df['BUSINESS_NEGATIVE'] = df['BUSINESS'].swifter.apply(negative_word)
df['RISK FACTORS_NEGATIVE'] = df['RISK FACTORS'].swifter.apply(negative_word)
df['SELECTED FINANCIAL DATA_NEGATIVE'] = df['SELECTED FINANCIAL DATA'].swifter.apply(negative_word)
df['MDA_NEGATIVE'] = df['MDA'].swifter.apply(negative_word)
df['FSS_NEGATIVE'] = df['FSS'].swifter.apply(negative_word)


Wall time: 3min 36s


In [66]:
%%time
df['BUSINESS_UNCERTAINTY'] = df['BUSINESS'].swifter.apply(uncertainty_score)
df['RISK FACTORS_UNCERTAINTY'] = df['RISK FACTORS'].swifter.apply(uncertainty_score)
df['SELECTED FINANCIAL DATA_UNCERTAINTY'] = df['SELECTED FINANCIAL DATA'].swifter.apply(uncertainty_score)
df['MDA_UNCERTAINTY'] = df['MDA'].swifter.apply(uncertainty_score)
df['FSS_UNCERTAINTY'] = df['FSS'].swifter.apply(uncertainty_score)


Wall time: 45.6 s


In [67]:
%%time
df['BUSINESS_average_sentence_length'] = df['BUSINESS'].swifter.apply(average_sentence_length)
df['RISK_average_sentence_length'] = df['RISK FACTORS'].swifter.apply(average_sentence_length)
df['SELECTED_average_sentence_length'] = df['SELECTED FINANCIAL DATA'].swifter.apply(average_sentence_length)
df['MDA_average_sentence_length'] = df['MDA'].swifter.apply(average_sentence_length)
df['FSS_average_sentence_length'] = df['FSS'].swifter.apply(average_sentence_length)


Wall time: 27.3 s


In [68]:
%%time
df['BUSINESS_word_count'] = df['BUSINESS'].swifter.apply(total_word_count)
df['RISK_word_count'] = df['RISK FACTORS'].swifter.apply(total_word_count)
df['SELECTED_word_count'] = df['SELECTED FINANCIAL DATA'].swifter.apply(total_word_count)
df['MDA_word_count'] = df['MDA'].swifter.apply(total_word_count)
df['FSS_word_count'] = df['FSS'].swifter.apply(total_word_count)


Wall time: 17.3 s


In [69]:
%%time
df['BUSINESS_polarity_score'] = np.vectorize(polarity_score)(df['BUSINESS_POSITIVE'],df['BUSINESS_NEGATIVE'])
df['RISK_polarity_score'] = np.vectorize(polarity_score)(df['RISK FACTORS_POSITIVE'],df['RISK FACTORS_NEGATIVE'])
df['SELECTED_polarity_score'] = np.vectorize(polarity_score)(df['SELECTED FINANCIAL DATA_POSITIVE'],df['SELECTED FINANCIAL DATA_NEGATIVE'])
df['MDA_polarity_score'] = np.vectorize(polarity_score)(df['MDA_POSITIVE'],df['MDA_NEGATIVE'])
df['FSS_polarity_score'] = np.vectorize(polarity_score)(df['FSS_POSITIVE'],df['FSS_NEGATIVE'])

Wall time: 6.01 ms


In [70]:
%%time
df['BUSINESS_percentage_of_complex_words'] = df['BUSINESS'].swifter.apply(percentage_complex_word)
df['RISK_percentage_of_complex_words'] =   df['RISK FACTORS'].swifter.apply(percentage_complex_word)
df['SELECTED_percentage_of_complex_words'] = df['SELECTED FINANCIAL DATA'].swifter.apply(percentage_complex_word)
df['MDA_percentage_of_complex_words'] = df['MDA'].swifter.apply(percentage_complex_word)
df['FSS_percentage_of_complex_words'] = df['FSS'].swifter.apply(percentage_complex_word)


Wall time: 25.1 s


In [71]:
%%time
df['BUSINESS_complex_word_count'] = df['BUSINESS'].swifter.apply(complex_word_count)
df['RISK_complex_word_count'] =   df['RISK FACTORS'].swifter.apply(complex_word_count)
df['SELECTED_complex_word_count'] = df['SELECTED FINANCIAL DATA'].swifter.apply(complex_word_count)
df['MDA_complex_word_count'] = df['MDA'].swifter.apply(complex_word_count)
df['FSS_complex_word_count'] = df['FSS'].swifter.apply(complex_word_count)


Wall time: 25.9 s


In [72]:
%%time
df['BUSINESS_fog_index'] = np.vectorize(fog_index)(df['BUSINESS_average_sentence_length'],df['BUSINESS_percentage_of_complex_words'])
df['RISK_fog_index'] = np.vectorize(fog_index)(df['RISK_average_sentence_length'],df['RISK_percentage_of_complex_words'])
df['SELECTED_fog_index'] = np.vectorize(fog_index)(df['SELECTED_average_sentence_length'],df['SELECTED_percentage_of_complex_words'])
df['MDA_fog_index'] = np.vectorize(fog_index)(df['MDA_average_sentence_length'],df['MDA_percentage_of_complex_words'])
df['FSS_fog_index'] = np.vectorize(fog_index)(df['FSS_average_sentence_length'],df['FSS_percentage_of_complex_words'])

Wall time: 7.98 ms


In [73]:
df['BUSINESS_Pos_Neg_polarity'] = df['BUSINESS_polarity_score'].swifter.apply(lambda x: 1 if x>0 else 0)
df['RISK_Pos_Neg_polarity'] = df['RISK_polarity_score'].swifter.apply(lambda x: 1 if x>0 else 0)
df['SELECTED_Pos_Neg_polarity'] = df['SELECTED_polarity_score'].swifter.apply(lambda x: 1 if x>0 else 0)
df['MDA_Pos_Neg_polarity'] = df['MDA_polarity_score'].swifter.apply(lambda x: 1 if x>0 else 0)
df['FSS_Pos_Neg_polarity'] = df['FSS_polarity_score'].swifter.apply(lambda x: 1 if x>0 else 0)

In [74]:
df.head()

,ticker,cik,filing_date,id,BUSINESS,RISK FACTORS,SELECTED FINANCIAL DATA,MDA,FSS,BUSINESS_Tokens,...,BUSINESS_fog_index,RISK_fog_index,SELECTED_fog_index,MDA_fog_index,FSS_fog_index,BUSINESS_Pos_Neg_polarity,RISK_Pos_Neg_polarity,SELECTED_Pos_Neg_polarity,MDA_Pos_Neg_polarity,FSS_Pos_Neg_polarity
0,GD,40533,2/17/2012,0,overview general dynamic aerospace defense com...,investment common stock debt security subject ...,following table present selected historicalnfi...,based consolidated financial statement prepare...,consolidated statement earningsn year ended de...,"[overview, general, dynamic, aerospace, defens...",...,3305.797047,74.591398,78.958376,408.606856,3260.200785,0,0,0,0,0
1,MTB,36270,3/19/1999,1,nstatistical disclosure pursuant guide distrib...,c risk elementsn nonaccrual pastdue renegotiat...,selected consolidated yearend balance b consol...,company international activity discussed note ...,financial statement schedule required inapplic...,"[nstatistical, disclosure, pursuant, guide, di...",...,648.580876,219.778506,36.567033,4.966667,13.776471,0,0,0,0,0
2,RRC,315852,3/6/2001,2,n ngeneralnrange resource corporation range en...,ovides financing small oil gas producer thenpu...,n nthe following table present selected financ...,capitalized term herein defined footnote thenc...,nreference made index financial statement page...,"[n, ngeneralnrange, resource, corporation, ran...",...,1619.397233,322.581141,70.182857,830.192000,55.431884,0,1,1,0,0
3,LH,920148,2/26/2009,3,clinical laboratory testing industry competiti...,risk associated company businesschanges federa...,presented caption statement operation data bal...,included elsewhere hereiyear ended december b ...,reference made index page f financial report i...,"[clinical, laboratory, testing, industry, comp...",...,2857.805711,1043.807436,304.521419,287.716273,3.733333,0,0,0,0,0
4,SYY,96021,9/16/2004,4,n overview sysco corporation acting subsidiary...,nitem quantitative qualitative disclosure mark...,n fiscal yearn n n week n n thousand except sh...,mical product distributornreed distributor inc...,sysco corporation subsidiary index consolidate...,"[n, overview, sysco, corporation, acting, subs...",...,569.404357,308.199481,54.535294,131.798176,37.782979,1,0,0,0,0


In [75]:
df.columns

Index(['ticker', 'cik', 'filing_date', 'id', 'BUSINESS', 'RISK FACTORS',
       'SELECTED FINANCIAL DATA', 'MDA', 'FSS', 'BUSINESS_Tokens',
       'RISK_Tokens', 'SELECTED_Tokens', 'MDA_Tokens', 'FSS_Tokens',
       'BUSINESS_POSITIVE', 'RISK FACTORS_POSITIVE',
       'SELECTED FINANCIAL DATA_POSITIVE', 'MDA_POSITIVE', 'FSS_POSITIVE',
       'BUSINESS_NEGATIVE', 'RISK FACTORS_NEGATIVE',
       'SELECTED FINANCIAL DATA_NEGATIVE', 'MDA_NEGATIVE', 'FSS_NEGATIVE',
       'BUSINESS_UNCERTAINTY', 'RISK FACTORS_UNCERTAINTY',
       'SELECTED FINANCIAL DATA_UNCERTAINTY', 'MDA_UNCERTAINTY',
       'FSS_UNCERTAINTY', 'BUSINESS_average_sentence_length',
       'RISK_average_sentence_length', 'SELECTED_average_sentence_length',
       'MDA_average_sentence_length', 'FSS_average_sentence_length',
       'BUSINESS_word_count', 'RISK_word_count', 'SELECTED_word_count',
       'MDA_word_count', 'FSS_word_count', 'BUSINESS_polarity_score',
       'RISK_polarity_score', 'SELECTED_polarity_score', 'MDA_p

In [76]:
%%time
#df.to_csv("/content/drive/My Drive/PHD/Final_Test_OutPut.csv",index=False)
df.to_csv("Final_Test_OutPut.csv",index=False)

Wall time: 3.73 s
